# Thread Pool

### Проблема с созданием тредов
* Дорого создавать новый системный поток
* GC спасибо вам за это не скажет

### Поэтому придумали thread pool
*** Рассказать про object pool ***

![alt text](threadpool0.png)

## Процесс выполнения задач на тредпуле

![alt text](threadpool1.png)

![alt text](threadpool2.png)

![alt text](threadpool3.png)

![alt text](threadpool4.png)

### В данном случает тут все как в местной пятерочке:
* Есть вероятность, что будет создан новый системный тред
* Но скорее всего она просто будет торчать в очереде пока воркер не освободится
* Если тасок мало, то воркеры могут в общем-то уйти из тредпула

### Нужно ли работать с тредпулом?
* Нет, существуют более удобные абстракции для работы с тредпулом
* А если очень хочется там потыкать? Можете попробовать, но скорее всего ваш код будет работать медленнее


# Async / Await

### Предположим, мы пишем свой http сервер

```c#
void ProcessRequestNaive(Listener listener)
{
  while(true) 
  {
    var socket = Socket.Accept(listener);   // Сидим ждем пока постучатся

    var recieveThread = Thread.Start(() =>  // Дождались пользователя и создали тред
    {
      var data = socket.Recieve();          // Делаем всю логику
      var processedData = Process(data);
      socket.Send(processedData);
    });
  }
}
```

### Можно сделать чуть лучше

```c#
void ProcessRequestNaive(Listener listener)
{
  while(true) 
  {
    var socket = Socket.Accept(listener);   // Сидим ждем пока постучатся (а ждать можно долго)

    var recieveThread = ThreadPool.QueueUserWorkItem(() =>  // Дождались пользователя и взяли тред из тредпула
    {
      var data = socket.Recieve();          // Достать данные из сокета тоже не быстрое дело
      var processedData = Process(data);
      socket.Send(processedData);           // Впрочем, как и запихать их обратно
    });
  }
}
```

### Но есть следующая проблема - мы впустую тратим время на синхронное ожидание

### Для решения этой проблемы создали механизм ассинхронного выполнения

```c#
Task ProcessRequestAsync(Listener listener)
{
  while(true) 
  {
    var socket = await Socket.AcceptAsync(listener);  // Теперь не надо ждать и впустую тратить время процессора

    var data = await socket.RecieveAsync();           
    var processedData = Process(data);

    await socket.Send(processedData);                 
  }
}
```

## Как это работает под капотом

![alt text](async_workflow.png)

## Async State Machine


Взято отсюда: https://habr.com/ru/articles/470830/

```c#
public static async Task Delays()
{
     Console.WriteLine(1);
     await Task.Delay(1000);
     Console.WriteLine(2);
     await Task.Delay(1000);
     Console.WriteLine(3);
}
```

### Конвертируется компилятором в такой метод (уже без async)

```c#
[AsyncStateMachine(typeof(DelaysStateMachine))]
public Task Delays()
{
     DelaysStateMachine stateMachine = new DelaysStateMachine();
     stateMachine.taskMethodBuilder = AsyncTaskMethodBuilder.Create();
     stateMachine.currentState = -1;
     AsyncTaskMethodBuilder builder = stateMachine.taskMethodBuilder;
     taskMethodBuilder.Start(ref stateMachine);
     return stateMachine.taskMethodBuilder.Task;
}
```

## Сама ассинхронная машина состояний

```c#
private sealed class DelaysStateMachine : IAsyncStateMachine
{
    //откражает текущее состояние, то есть на каком await выполняется ожидание
    //так возможно восстановление выполнения метода с любого await'a
    public int currentState; 
    public AsyncTaskMethodBuilder taskMethodBuilder;
    //текущий объект ожидания
    private TaskAwaiter taskAwaiter;

    //все параметры метода, как и локальные переменные сохраняются в поля для сохранения между ожиданиями при "отпускании" потока
    public int paramInt;
    private int localInt;

    private void MoveNext()
    {
        int num = currentState;
        
        try
        {
            TaskAwaiter awaiter2;
            TaskAwaiter awaiter1;

            switch (num)
            {
                default:
                    localInt = paramInt;  //до первого await
                    Console.WriteLine(1);  //до первого await
                    awaiter1 = Task.Delay(1000).GetAwaiter();  //до первого await
                    if (!awaiter1.IsCompleted) // сам await. Проверяется, завершился ли метод
                    {
                        num = (currentState = 0); //обновление состояние, чтобы возобновить с нужного места
                        taskAwaiter = awaiter1; //сохранение из локальной в поле, ведь после возобновления локальные переменные не сохранятся
                        DelaysStateMachine stateMachine = this; //чтобы передать по ссылке
                        taskMethodBuilder.AwaitUnsafeOnCompleted(ref awaiter1, ref stateMachine); // за страшным названием скрывается
                                                                                                  // лишь действия для присоединения к 
                                                                                                  // объекту ожидания продолжения в виде этого метода
                        return;
                    }
                    goto Il_AfterFirstAwait; //если метод завершен, переходим к коду, который следовал далее

                case 0:
                    awaiter1 = taskAwaiter; //восстанавливаем объект ожидания
                    taskAwaiter = default(TaskAwaiter); //обнуляем поле класса
                    num = (currentState = -1); //обновляем состояние
                    goto Il_AfterFirstAwait; //переходим непосредственно к логике из исходного метода
                case 1: // мы здесь, если вторая операция не завершилась сразу, а было присоеденино продолжение, которое как раз и запустилось.
                    awaiter2 = taskAwaiter;
                    taskAwaiter = default(TaskAwaiter);
                    num = (currentState = -1);
                    goto Il_AfterSecondAwait;
                case 2:
                    {
                        awaiter = taskAwaiter;
                        taskAwaiter = default(TaskAwaiter);
                        num = (currentState = -1);
                        break;
                    }

                    Il_AfterFirstAwait: //если мы здесь, то первая операция завершилась так или иначе
                    awaiter1.GetResult(); //соответсвенно результат доступен и мы его получаем
                    Console.WriteLine(2); //выполнение того кода, который шел после первого await
                    awaiter2 = Task.Delay(1000).GetAwaiter(); //Выполнение второй асинхронной операции
                    
                    if (!awaiter2.IsCompleted) 
                    {
                        num = (currentState = 1);
                        taskAwaiter = awaiter4;
                        DelaysStateMachine stateMachine = this;
                        taskMethodBuilder.AwaitUnsafeOnCompleted(ref awaiter4, ref stateMachine);
                        return;
                    }

                    Il_AfterSecondAwait:
                    awaiter2.GetResult();
                    Console.WriteLine(3); //код после второй асинхронной операции
            }

            awaiter.GetResult();
        }
        catch (Exception exception)
        {
            currentState = -2;         // -2 - код,сигнализирующий окончание метода (в данном случае это значит было исключение)
            taskMethodBuilder.SetException(exception);
            return;
        }
        currentState = -2;             // -2 - код, сигнализирующий окончание метода (в данном случае это значит что все ок)
        taskMethodBuilder.SetResult(); //если бы использовали асинхронные операции, которые возвращают результат, он был бы параметром этого метода
    }
}
```

### Объясняю на пальцах

![alt text](shizophrenia.png)

## Контексты синхронизации и их разница
* Thread Pool
* WPF/WinForms Dispatcher

## Подковырки в ассинхронных задачах

* async void и исключения

* Wait async в однопоточных контекстах синхронизации

```c#
void OnButtonClickSync(object sender, EventArgs e)
{
  Console.WriteLine("Waiting")
  DelayAsync().Wait();
  Console.WriteLine("Performed")
}

async Task DelayAsync()
{
  await Task.Delay(2000);
}
```

* Task vs ValueTask

## Как делать свои собственные ассинхронные задачи



```c#
// Метод срабатывает, когда наступает очередь игрока
public Task MakeNextTurn()
{
    AddActionEvents();

    return _actionCompletionSource.Task;
}

private void AddActionEvents()
{
    // Группа ивентов на срабатывания нажатия клавиш
    _playerMoveInput.MoveForwardAction.action.performed += OnMoveForward;
    _playerMoveInput.MoveBackwardAction.action.performed += OnMoveBackward;
    _playerMoveInput.MoveLeftAction.action.performed += OnMoveLeft;
    _playerMoveInput.MoveRightAction.action.performed += OnMoveRight;
}

private void RemoveActionEvents()
{
    // Не забываем отписываться от ивентов
    _playerMoveInput.MoveForwardAction.action.performed -= OnMoveForward;
    _playerMoveInput.MoveBackwardAction.action.performed -= OnMoveBackward;
    _playerMoveInput.MoveLeftAction.action.performed -= OnMoveLeft;
    _playerMoveInput.MoveRightAction.action.performed -= OnMoveRight;
}

// Когда игрок нажимает на клавиши, то ивенты стреляют и вызывают этот метод
private void Move(int xOffset, int yOffset, int zOffset)
{
    var offsetVector = new Vector3Int(xOffset, yOffset, zOffset);
    
    if (_playerEntity.TryMoveToNextCell(offsetVector, _levelModel))
    {
        RemoveActionEvents();
        _playerAnimator.Move(offsetVector);

        if (_actionCompletionSource.TrySetResult() == false) // Сигнализируем и выставлям результат, что таска завершилась
                                                             // После этого срабатывает MoveNext(), и выполнение продолжается
                                                             // на контексте синхронизации
        {
            _loggingService.LogError("Cannot set task result on completion");
        }
    }
}
```